# Getting started

Once you've chosen your scenario, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [1]:
import pandas as pd
import numpy as np
## Load the data into a DataFrame
df = pd.read_csv('../Resources/Iowa_Liquor_sales_sample_10pct.csv')

df.head()

,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0,5.55


In [2]:
cc = df.set_index('City')['County'].to_frame()
cc.dropna(inplace=True)
undefined = pd.DataFrame([['Wayne'],['Polk'],['Fremont'],['Wayne'],['Polk'],['Fremont']], columns=['County'], index=['SEYMOUR', 'RUNNELLS', 'TABOR','SEYMOUR', 'RUNNELLS', 'TABOR'])
df2 = pd.concat([cc, undefined])

def get_county(city):
    return df2.T[city].values[0][0]


In [3]:
citys_wo_county = df[pd.isnull(df["County"])]["City"].to_frame()
for row in citys_wo_county.index:
    df.set_value(row, 'County', get_county(citys_wo_county.ix[row]['City']))

In [4]:
cat = df.set_index('Category')['Category Name'].to_frame()
cat.dropna(inplace=True)
undefined_cats = pd.DataFrame([["Moonshine"],["Reposado"],["Vodka"],["Hennessy"],["Spiced Rum"],["Rum"],["Amaretto"],["Spirit"],["Moonshine"],["Moonshine"],["Rum"]], columns=['Category Name'], index=['1011700.0', '1022200.0', '1032100.0', '1052100.0', '1062400.0', '1062500.0', '1082100.0', '1082200.0', '1091100.0', '1091300.0', '1092100.0'])
all_cats = pd.concat([cat, undefined_cats])

def get_category_name(category):
    return all_cats.T[str(float(category))].values[0]

In [5]:
category_wo_name = df[pd.isnull(df["Category Name"])]["Category"].to_frame()

for row in category_wo_name.index:
    if not np.isnan(category_wo_name.ix[row]['Category']):
        df.set_value(row, 'Category Name', get_category_name(category_wo_name.ix[row]['Category']))

In [15]:
del df['County Number']
df = df.dropna()

In [7]:
df["Date"] = pd.to_datetime(df["Date"])

In [8]:
del df['Category']

In [9]:
df['State Bottle Cost'] = df['State Bottle Cost'].apply(lambda x: x.replace('$','')).astype(float)
df['State Bottle Retail'] = df['State Bottle Retail'].apply(lambda x: x.replace('$','')).astype(float)
df['Sale (Dollars)'] = df['Sale (Dollars)'].apply(lambda x: x.replace('$','')).astype(float)

In [10]:
prob_zip = []
for x in df['Zip Code']:
    if x[0] != '5':
        if x not in prob_zip:
            prob_zip.append(x)
        else:
            pass
    else:
        pass
df[df['Zip Code'] == prob_zip[0]]['City'].unique()
#dunlap has incorrect zip code, correct zip code is 51529
df['Zip Code'] = df['Zip Code'].apply(lambda x: x.replace('712-2','51529'))
#zipcodes are fixed

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270887 entries, 0 to 270954
Data columns (total 16 columns):
Date                     270887 non-null datetime64[ns]
Store Number             270887 non-null int64
City                     270887 non-null object
Zip Code                 270887 non-null object
County                   270887 non-null object
Category Name            270887 non-null object
Vendor Number            270887 non-null int64
Item Number              270887 non-null int64
Item Description         270887 non-null object
Bottle Volume (ml)       270887 non-null int64
State Bottle Cost        270887 non-null float64
State Bottle Retail      270887 non-null float64
Bottles Sold             270887 non-null int64
Sale (Dollars)           270887 non-null float64
Volume Sold (Liters)     270887 non-null float64
Volume Sold (Gallons)    270887 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(5), object(5)
memory usage: 35.1+ MB


# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [12]:
import seaborn as sns
import matplotlib.pyplot as plt

## Record your findings

Be sure to write out anything observations from your exploratory analysis.

# Mine the data
Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to
compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations [here](http://pandas.pydata.org/pandas-docs/stable/groupby.html) for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.

# Refine the data
Look for any statistical relationships, correlations, or other relevant properties of the dataset.

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [13]:
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score

In [77]:
best_counties = pd.DataFrame(columns=['county', 'r2'])

In [97]:
def change_to_float(inpt, replace):
    if type(inpt) == float:
        return inpt
    else:
        return inpt.replace(replace,'')

In [102]:
IA = pd.read_csv('../Resources/IA_Data_City_Master.csv')

IA['Population'] = IA['Population'].apply(lambda x: change_to_float(x,',')).astype(float)
IA['Male'] = IA['Male'].apply(lambda x: change_to_float(x,',')).astype(float)
IA['Female'] = IA['Female'].apply(lambda x: change_to_float(x,',')).astype(float)
IA['Median Age'] = IA['Median Age'].apply(lambda x: change_to_float(x,',')).astype(float)
IA['Per Capita Income'] = IA['Per Capita Income'].apply(lambda x: change_to_float(x,',')).astype(float)

IA.head()

,City,Population,Male,Female,Median Age,Household Income,Per Capita Income,House Value,Zip Codes
0,STATE AVERAGE,NaN,NaN,NaN,38.0,52229,NaN,126900,NaN
1,BURLINGTON,25539.0,12149.0,13390.0,42.1,38614,23850.0,85042,52601
2,FORT DODGE,24594.0,12606.0,11988.0,38.9,37247,23320.0,81061,50501
3,CARROLL,10007.0,4664.0,5343.0,40.5,43807,25017.0,125573,51401
4,FAIRFIELD,9750.0,4804.0,4946.0,43.8,46013,25388.0,87391,"52556, 52557"


In [104]:
IA['mf'] = IA['Male']/IA['Female']
IA.drop(['Male', 'Female'], axis=1, inplace=True)
IA.head()

,City,Population,Median Age,Household Income,Per Capita Income,House Value,Zip Codes,mf
0,STATE AVERAGE,NaN,38.0,52229,NaN,126900,NaN,NaN
1,BURLINGTON,25539.0,42.1,38614,23850.0,85042,52601,0.907319
2,FORT DODGE,24594.0,38.9,37247,23320.0,81061,50501,1.051552
3,CARROLL,10007.0,40.5,43807,25017.0,125573,51401,0.872918
4,FAIRFIELD,9750.0,43.8,46013,25388.0,87391,"52556, 52557",0.971290


In [91]:
test = df[df['County'] == 'Adams']
df_X = test.drop(["Sale (Dollars)","Item Description","Category Name", "City", "Zip Code", "County", "Date", "Volume Sold (Liters)"], axis=1)
df_y = test["Sale (Dollars)"]
# X = df_X.merge(IA[IA['County'] == 'Adams'])
# y = df_y
df_X
IA[IA['County'] == 'Adams']


,County,Per Capita Yearly Income,Housing Estimate,Population Estimate
1,Adams,"23,707",2007,3796


In [92]:
df_X

,Store Number,Vendor Number,Item Number,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Volume Sold (Gallons)
2293,2327,260,11298,1750,31.99,47.99,2,0.92
4704,2327,55,21597,1000,4.94,7.41,1,0.26
5658,2327,260,77683,750,8.25,12.38,2,0.40
7245,2327,65,71441,500,15.83,23.75,1,0.13
7364,2327,305,77573,750,7.94,11.91,3,0.59
8036,2327,421,64866,750,8.98,13.47,2,0.40
8118,2327,370,42717,1000,10.49,15.74,3,0.79
8977,2327,35,43123,200,2.50,3.75,3,0.16
11020,2327,192,65253,200,3.51,5.27,3,0.16
12388,2327,260,65061,600,15.63,23.45,1,0.16


In [83]:
# set the X matrix/DataFrame (IVs) and use the feature names as cols
for county in df['County'].unique():
    test = df[df['County'] == county]
    df_X = test.drop(["Sale (Dollars)","Item Description","Category Name", "City", "Zip Code", "County", "Date", "Volume Sold (Liters)"], axis=1)
    df_y = test["Sale (Dollars)"]
    X = df_X.merge(IA[IA['County'] == county])
    y = df_y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33)
    lasso_tts = linear_model.Lasso(alpha=.01)
    tts_model = lasso_tts.fit(X_train, y_train)

    tts_lasso_r2 = tts_model.predict(X_test)
#     r2_score(y_true=y_test, y_pred=tts_model.predict(X_test))
    best_counties.loc[len(best_counties)] = [county, tts_lasso_r2]

    #     print county, 'lasso r2:     ', tts_lasso_r2
best_counties

,county,r2
0,Bremer,0.909547
1,Scott,0.730775
2,Black Hawk,0.695716
3,Story,0.811604
4,Wright,0.846273
5,Linn,0.813500
6,Wapello,0.747338
7,Cerro Gordo,0.758828
8,Polk,0.761011
9,Benton,-1.630723


In [109]:
heatmap = df.groupby('Zip Code')['Sale (Dollars)'].sum().to_frame()

In [110]:
heatmap.to_csv('heatmap.csv')

In [111]:
heatmap

,Sale (Dollars)
Zip Code,
50002,5851.20
50003,47716.10
50006,12280.24
50009,298406.69
50010,932695.71
50014,48778.93
50020,10402.47
50021,504166.37
50022,119981.49


## Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.